In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import json

with open('/content/drive/MyDrive/DS204- Đồ án KHDL/Data/data gốc ViQUAD/ViQuAD v2/UIT-ViQuAD 2.0/dev (public_test).json', 'r', encoding="utf-8") as f:
  dev = json.load(f)
dev=dev["data"]

with open('/content/drive/MyDrive/DS204- Đồ án KHDL/Data/data gốc ViQUAD/ViQuAD v2/UIT-ViQuAD 2.0/test (private_test).json', 'r', encoding="utf-8") as f:
  test = json.load(f)
test=test["data"]
import json

with open('/content/drive/MyDrive/DS204- Đồ án KHDL/Data/data gốc ViQUAD/ViQuAD v2/UIT-ViQuAD 2.0/train.json', 'r', encoding="utf-8") as f:
  train = json.load(f)
train=train["data"]

In [37]:
# chuyển json thành dataframe 
def json_to_df_test(data):
  passage=[]
  question=[]
  id=[]
  for i in range(len(data)):
   for j in data[i]["paragraphs"]:
    for k in j["qas"]:
        question.append(k['question'])
        id.append(k['id'])
        passage.append(j["context"])
  print(len(passage))
  print(len(question))
  print(len(id))

  dataframe=pd.DataFrame({"id": id, "passage": passage, "question": question})
  return dataframe


In [38]:
def json_to_df(data):
      passage=[]
      impossible=[]
      question=[]
      id=[]
      answer_Viquad=[]
      for i in range(len(data)):
        for j in data[i]["paragraphs"]:
          for k in j["qas"]:
            question.append(k['question'])
            impossible.append(k['is_impossible'])
            id.append(k['id'])
            passage.append(j["context"])
            answer_Viquad.append(k['answers'])
      print(len(passage))
      print(len(question))
      print(len(impossible))
      print(len(id))
      dataframe=pd.DataFrame({"id": id "passage": passage, "question": question, "Answer": answer_Viquad, , "is_impossible": impossible,})
      return dataframe

In [39]:
#lưu dataframe 

test=json_to_df_test(test)
test["Answer_new"]=np.nan
test["Question_type"]=np.nan
test["Reason_type"]=np.nan
test["new_question"]=np.nan
test["question_domain"]=np.nan
test.to_excel("/content/drive/MyDrive/DS204- Đồ án KHDL/Data/Data đã chuyển sang excel+csv/test.xlsx")

dev=json_to_df(dev)
dev["Answer_new"]=np.nan
dev["Question_type"]=np.nan
dev["Reason_type"]=np.nan
dev["new_question"]=np.nan
dev["question_domain"]=np.nan
dev.to_excel("/content/drive/MyDrive/DS204- Đồ án KHDL/Data/Data đã chuyển sang excel+csv/dev.xlsx")

train=json_to_df(train)
train["Answer_new"]=np.nan
train["Question_type"]=np.nan
train["Reason_type"]=np.nan
train["new_question"]=np.nan
train["question_domain"]=np.nan
train.to_excel("/content/drive/MyDrive/DS204- Đồ án KHDL/Data/Data đã chuyển sang excel+csv/train.xlsx")



7301
7301
7301
3821
3821
3821
3821
28457
28457
28457
28457


In [45]:
train_dev=pd.concat([train,dev])


In [ ]:
train_dev

In [53]:
train_dev.to_excel("/content/drive/MyDrive/DS204- Đồ án KHDL/Data/Data đã chuyển sang excel+csv/Full_data_baogom_impossible.xlsx")

In [51]:
full=full_data[full_data["is_impossible"]==False]

In [56]:
index=[]
for i in range(len(full)):
  index.append(i)
full["index"]=index

<ipython-input-56-02087151514b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full["index"]=index


In [57]:
full.to_excel("/content/drive/MyDrive/DS204- Đồ án KHDL/Data/Data đã chuyển sang excel+csv/full_ready.xlsx")

In [ ]:
ganthu=full.sample(150)
ganthu

In [60]:
full.shape

(21893, 11)

In [63]:
Tinh=full.sample(6500)
QA=full[~full.index.isin(Tinh.index)].sample(6500)
HM=full[~full.index.isin(pd.concat([Tinh,QA]).index)].sample(6500)

In [64]:
ganthu.to_excel("/content/drive/MyDrive/DS204- Đồ án KHDL/Data/Data đã chuyển sang excel+csv/data_gán/ganthu.xlsx")
Tinh.to_excel("/content/drive/MyDrive/DS204- Đồ án KHDL/Data/Data đã chuyển sang excel+csv/data_gán/Tinh.xlsx")
QA.to_excel("/content/drive/MyDrive/DS204- Đồ án KHDL/Data/Data đã chuyển sang excel+csv/data_gán/QA.xlsx")
HM.to_excel("/content/drive/MyDrive/DS204- Đồ án KHDL/Data/Data đã chuyển sang excel+csv/data_gán/HM.xlsx")

In [ ]:
import numpy as np

class BM25:
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75):
        self.corpus_size = len(corpus)
        self.avgdl = sum(len(doc) for doc in corpus) / self.corpus_size
        self.k1 = k1
        self.b = b
        
        self.tokenizer = tokenizer or self.tokenize_default
        self.doc_freqs = []
        self.idf = {}
        self.doc_len = []
        self.initialize(corpus)
        
    def tokenize_default(self, text):
        return text.split()
        
    def initialize(self, corpus):
        for doc in corpus:
            frequencies = {}
            tokens = self.tokenizer(doc)
            self.doc_len.append(len(tokens))
            for token in tokens:
                if token not in frequencies:
                    frequencies[token] = 0
                frequencies[token] += 1
            self.doc_freqs.append(frequencies)
            
            for token in frequencies:
                if token not in self.idf:
                    self.idf[token] = 0
                self.idf[token] += 1
                
        for token in self.idf:
            self.idf[token] = np.log(self.corpus_size - self.idf[token] + 0.5) - np.log(self.idf[token] + 0.5)
        
    def get_score(self, query, index):
        score = 0
        frequencies = self.doc_freqs[index]
        doc_len = self.doc_len[index]
        for token in self.tokenizer(query):
            if token in frequencies:
                idf = self.idf[token] if token in self.idf else 0
                score += (idf * frequencies[token] * (self.k1 + 1)
                          / (frequencies[token] + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score
        
    def get_scores(self, query):
        scores = np.zeros(self.corpus_size)
        for i in range(self.corpus_size):
            scores[i] = self.get_score(query, i)
        return scores



In [ ]:
def get_passage( data):
  print(len(data))
  a = data["passage"].values
  dict_a = OrderedDict()
  for i in a:
      dict_a[i] = dict_a.get(i, 0) + 1
  b = list(dict_a.keys())  
  return b

In [ ]:
def get_passage_relevant(data):
  # lấy đoạn văn
      passage=get_passage(data)
      print(passage)
      context_max1=[]
      context_max2=[]
      bm25 = BM25(passage)
      for i in data["question"]:
        scores_i = bm25.get_scores(i)
        #list_score.append(scores_i)
        arr = scores_i
      # Sử dụng argsort để sắp xếp mảng theo thứ tự giảm dần và lấy chỉ số của hai phần tử đầu tiên
        sorted_indices = np.argsort(-arr)
        max_indices = sorted_indices[:2]
        #print(max_indices)
        max_1=max_indices[0]
        max_2=max_indices[1]
        context_max1.append(passage[max_1])
        context_max2.append(passage[max_2])
      data["context_max1"]=context_max1
      data["context_max2"]=context_max2
      return data

In [ ]:
'''test=get_passage_relevant(test)
test.to_excel("/content/drive/MyDrive/DS204- Đồ án KHDL/Data/Data đã chuyển sang excel+csv/test_bm25.xlsx")

dev=get_passage_relevant(dev)
dev.to_excel("/content/drive/MyDrive/DS204- Đồ án KHDL/Data/Data đã chuyển sang excel+csv/dev_bm25.xlsx")'''

train=get_passage_relevant(train)
train.to_excel("/content/drive/MyDrive/DS204- Đồ án KHDL/Data/Data đã chuyển sang excel+csv/train_bm25.xlsx")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained GloVe embeddings
embeddings_index = {}
with open('/content/drive/MyDrive/DS204- Đồ án KHDL/embedding /glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Define function to preprocess text
#def preprocess_text(text):
    # Remove stop words, punctuation, convert to lowercase
    # Implement this according to your needs
    #return processed_text

# Define function to calculate the average embedding of a sentence
def sentence_embedding(sentence, embeddings_index, dim=100):
    # Tokenize sentence and preprocess
    words = sentence.split()
    # Initialize empty array for sentence embedding
    sent_emb = np.zeros((dim,))
    # Count number of valid words in sentence (i.e., words in embeddings)
    count = 0
    for word in words:
        if word in embeddings_index:
            sent_emb += embeddings_index[word]
            count += 1
    if count > 0:
        # Take the average of word embeddings
        sent_emb /= count
    return sent_emb


In [ ]:
def GloVe(data)
# Define the question
    passage=get_passage(data)
    for i in data["question"]:
      list_similarity=[]
      question_emb = sentence_embedding(question, embeddings_index)
      for passage in passage:
        passage_emb = sentence_embedding(passage, embeddings_index)
    # Calculate cosine similarity between the question and passages
        similarity = cosine_similarity(question_emb.reshape(1,-1), passage_emb.reshape(1,-1))
        list_similarity.append(similarity)
        arr = list_similarity
    # Sử dụng argsort để sắp xếp mảng theo thứ tự giảm dần và lấy chỉ số của hai phần tử đầu tiên
      sorted_indices = np.argsort(-arr)
      max_indices = sorted_indices[:2]
      #print(max_indices)
      max_1=max_indices[0]
      max_2=max_indices[1]
      context_max1.append(passage[max_1])
      context_max2.append(passage[max_2])
    dev["context_max1"]=context_max1
    dev["context_max2"]=context_max2
    return data



In [ ]:
test=GloVe(test)
test.to_excel("/content/drive/MyDrive/DS204- Đồ án KHDL/Data/Data đã chuyển sang excel+csv/test_glove.xlsx")

dev=GloVe(dev)
dev.to_excel("/content/drive/MyDrive/DS204- Đồ án KHDL/Data/Data đã chuyển sang excel+csv/dev_glove.xlsx")

train=GloVe(train)
train.to_excel("/content/drive/MyDrive/DS204- Đồ án KHDL/Data/Data đã chuyển sang excel+csv/train_glove.xlsx")